In [1]:
import numpy as np
import pandas as pd

In [7]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path="eng_-french.csv"

In [9]:
#vectorize the data
input_texts=[]
target_texts=[]
input_characters= set()
target_characters= set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines= f.read().split('\n')
for line in  lines [: min(num_samples, len(lines) - 1)]:
    input_text = line.split('\t')
    target_text = line.split('\t')
    _ = line.split('\t')
    #we use 'tab' as the 'start sequence' character.
    #for the targets '\n' as 'end sequence' character.
    target_text =  '\t' + str(target_text) + '\n' 
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [10]:
target_texts

["\t['English words/sentences,French words/sentences']\n",
 "\t['Hi.,Salut!']\n",
 "\t['Run!,Cours\\u202f!']\n",
 "\t['Run!,Courez\\u202f!']\n",
 "\t['Who?,Qui ?']\n",
 "\t['Wow!,Ça alors\\u202f!']\n",
 "\t['Fire!,Au feu !']\n",
 '\t["Help!,À l\'aide\\u202f!"]\n',
 "\t['Jump.,Saute.']\n",
 "\t['Stop!,Ça suffit\\u202f!']\n",
 "\t['Stop!,Stop\\u202f!']\n",
 "\t['Stop!,Arrête-toi !']\n",
 "\t['Wait!,Attends !']\n",
 "\t['Wait!,Attendez !']\n",
 "\t['Go on.,Poursuis.']\n",
 "\t['Go on.,Continuez.']\n",
 "\t['Go on.,Poursuivez.']\n",
 "\t['Hello!,Bonjour !']\n",
 "\t['Hello!,Salut !']\n",
 "\t['I see.,Je comprends.']\n",
 '\t["I try.,J\'essaye."]\n',
 '\t["I won!,J\'ai gagné !"]\n',
 '\t["I won!,Je l\'ai emporté !"]\n',
 "\t['I won.,J’ai gagné.']\n",
 "\t['Oh no!,Oh non !']\n",
 "\t['Attack!,Attaque !']\n",
 "\t['Attack!,Attaquez !']\n",
 "\t['Cheers!,Santé !']\n",
 "\t['Cheers!,À votre santé !']\n",
 "\t['Cheers!,Merci !']\n",
 "\t['Cheers!,Tchin-tchin !']\n",
 "\t['Get up.,Lève-toi.']\n",

In [11]:
input_texts

[['English words/sentences,French words/sentences'],
 ['Hi.,Salut!'],
 ['Run!,Cours\u202f!'],
 ['Run!,Courez\u202f!'],
 ['Who?,Qui ?'],
 ['Wow!,Ça alors\u202f!'],
 ['Fire!,Au feu !'],
 ["Help!,À l'aide\u202f!"],
 ['Jump.,Saute.'],
 ['Stop!,Ça suffit\u202f!'],
 ['Stop!,Stop\u202f!'],
 ['Stop!,Arrête-toi !'],
 ['Wait!,Attends !'],
 ['Wait!,Attendez !'],
 ['Go on.,Poursuis.'],
 ['Go on.,Continuez.'],
 ['Go on.,Poursuivez.'],
 ['Hello!,Bonjour !'],
 ['Hello!,Salut !'],
 ['I see.,Je comprends.'],
 ["I try.,J'essaye."],
 ["I won!,J'ai gagné !"],
 ["I won!,Je l'ai emporté !"],
 ['I won.,J’ai gagné.'],
 ['Oh no!,Oh non !'],
 ['Attack!,Attaque !'],
 ['Attack!,Attaquez !'],
 ['Cheers!,Santé !'],
 ['Cheers!,À votre santé !'],
 ['Cheers!,Merci !'],
 ['Cheers!,Tchin-tchin !'],
 ['Get up.,Lève-toi.'],
 ['Go now.,"Va, maintenant."'],
 ['Go now.,Allez-y maintenant.'],
 ['Go now.,Vas-y maintenant.'],
 ["Got it!,J'ai pigé !"],
 ['Got it!,Compris !'],
 ['Got it?,Pigé\u202f?'],
 ['Got it?,Compris\u202f?']

In [12]:
input_characters

{'Are you winning?,Est-ce que vous êtes en train de gagner\xa0?',
 'Am I alone here?,Est-ce que je suis seul ici?',
 "We'll drive.,Nous conduirons.",
 'Is this fake?,Est-ce du chiqué ?',
 "He's loaded.,Il est plein aux as.",
 "I was burned.,J'ai été brûlée.",
 "It's silent.,C'est silencieux.",
 "You're gross!,Tu es malpoli !",
 'No one noticed.,Personne ne le remarqua.',
 "Don't be angry.,Ne sois pas fâché.",
 "We're a couple.,Nous sommes en couple.",
 "I'm hiding.,Je me cache.",
 "I'll try.,J'essaierai.",
 'We have to go.,Il nous faut partir.',
 "I like honey.,J'aime le miel.",
 'I want to talk.,Je veux discuter.',
 'Stay in bed.,Restez alités !',
 "I didn't care.,Je n'ai pas fait attention.",
 "I'm an orphan.,Je suis orphelin.",
 'I need it.,Il me le faut.',
 "Tom felt sick.,Tom se sentit mal à l'aise.",
 "We'll help you.,Nous vous aiderons.",
 "We're stunned.,Nous sommes sidérés.",
 'Did you go out?,Êtes-vous sortis ?',
 "I like oysters.,J'aime les huîtres.",
 'I just moved.,Je vien

In [13]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 '[',
 '\\',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '’'}

In [14]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [15]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 10000
Number of unique output tokens: 99
Max sequence length for inputs: 1
Max sequence length for outputs: 77


In [16]:
input_token_index=dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index=dict(
[(char,i) for i, char in enumerate(target_characters)])

In [17]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [22]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

### Definning the Encoder and Decoder.

In [23]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [24]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 153s 19ms/sample - loss: 1.9337 - accuracy: 0.5863 - val_loss: 1.7407 - val_accuracy: 0.5558
Epoch 2/100
8000/8000 [==============================] - 125s 16ms/sample - loss: 1.4279 - accuracy: 0.6274 - val_loss: 1.3989 - val_accuracy: 0.6239
Epoch 3/100
8000/8000 [==============================] - 99s 12ms/sample - loss: 1.1423 - accuracy: 0.6920 - val_loss: 1.2263 - val_accuracy: 0.6489
Epoch 4/100
8000/8000 [==============================] - 112s 14ms/sample - loss: 0.9543 - accuracy: 0.7267 - val_loss: 1.0348 - val_accuracy: 0.6934
Epoch 5/100
8000/8000 [==============================] - 115s 14ms/sample - loss: 0.8637 - accuracy: 0.7481 - val_loss: 0.9631 - val_accuracy: 0.7137
Epoch 6/100
8000/8000 [==============================] - 116s 15ms/sample - loss: 0.7990 - accuracy: 0.7649 - val_loss: 0.9189 - val_accuracy: 0.7237
Epoch 7/100
8000/8000 [==============================

8000/8000 [==============================] - 89s 11ms/sample - loss: 0.1182 - accuracy: 0.9679 - val_loss: 1.2553 - val_accuracy: 0.7579
Epoch 56/100
8000/8000 [==============================] - 88s 11ms/sample - loss: 0.1139 - accuracy: 0.9687 - val_loss: 1.2889 - val_accuracy: 0.7560
Epoch 57/100
8000/8000 [==============================] - 89s 11ms/sample - loss: 0.1090 - accuracy: 0.9704 - val_loss: 1.2997 - val_accuracy: 0.7580
Epoch 58/100
8000/8000 [==============================] - 92s 12ms/sample - loss: 0.1049 - accuracy: 0.9718 - val_loss: 1.2871 - val_accuracy: 0.7590
Epoch 59/100
8000/8000 [==============================] - 125s 16ms/sample - loss: 0.1009 - accuracy: 0.9728 - val_loss: 1.3099 - val_accuracy: 0.7590
Epoch 60/100
8000/8000 [==============================] - 126s 16ms/sample - loss: 0.0971 - accuracy: 0.9738 - val_loss: 1.3271 - val_accuracy: 0.7566
Epoch 61/100
8000/8000 [==============================] - 122s 15ms/sample - loss: 0.0929 - accuracy: 0.9753 - 

In [25]:
model.save('eng2french.h5')

In [29]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [30]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [31]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['English words/sentences,French words/sentences']
Decoded sentence: ['English words.,Bonnurez à lé basse.']

-
Input sentence: ['Hi.,Salut!']
Decoded sentence: ['Hi.,Salut!']

-
Input sentence: ['Run!,Cours\u202f!']
Decoded sentence: ['Run!,Cours\u202f!']

-
Input sentence: ['Run!,Courez\u202f!']
Decoded sentence: ['Run!,Courez\u202f!']

-
Input sentence: ['Who?,Qui ?']
Decoded sentence: ['Who?,Qui ?']

-
Input sentence: ['Wow!,Ça alors\u202f!']
Decoded sentence: ['Wow!,Ça alors\u202f!']

-
Input sentence: ['Fire!,Au feu !']
Decoded sentence: ['Fire!,Au feu !']

-
Input sentence: ["Help!,À l'aide\u202f!"]
Decoded sentence: ["Help!,À l'aide\u202f!"]

-
Input sentence: ['Jump.,Saute.']
Decoded sentence: ['Jump.,Saute.']

-
Input sentence: ['Stop!,Ça suffit\u202f!']
Decoded sentence: ['Stop!,Ça suffit\u202f!']

-
Input sentence: ['Stop!,Stop\u202f!']
Decoded sentence: ['Stop!,Stop\u202f!']

-
Input sentence: ['Stop!,Arrête-toi !']
Decoded sentence: ['Stop!,Arrête-toi !'

-
Input sentence: ['Be nice.,Soyez gentilles !']
Decoded sentence: ['Be nice.,Soyez gentilles !']

-
Input sentence: ['Beat it.,Dégage\u202f!']
Decoded sentence: ['Beat it.,Dégage\u202f!']

-
Input sentence: ['Call me.,Appelle-moi !']
Decoded sentence: ['Call me.,Appelle-moi !']

-
Input sentence: ['Call me.,Appellez-moi !']
Decoded sentence: ['Call me.,Appellez-moi !']

-
Input sentence: ['Call us.,Appelle-nous !']
Decoded sentence: ['Call us.,Appelle-nous !']

-
Input sentence: ['Call us.,Appelez-nous !']
Decoded sentence: ['Call us.,Appelez-nous !']

-
Input sentence: ['Come in.,Entrez\u202f!']
Decoded sentence: ['Come in.,Entrez\u202f!']

-
Input sentence: ['Come in.,Entre.']
Decoded sentence: ['Come in.,Entre.']

